In [2]:
import pandas as pd
import numpy as np

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report

In [8]:
data_path = "Telco-Customer-Churn 2.csv"

In [9]:
df = pd.read_csv(data_path)

In [10]:
print("First 5 rows of the data:")
print(df.info())

First 5 rows of the data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling

In [11]:
print("\n basic info:")
print(df.info())


 basic info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-n

In [12]:
target_column = 'Churn'

In [13]:
if df [target_column].dtype == '0':
    df[target_column] = df[target_column].map({'Yes':1,'No':0})

In [14]:
df = df.dropna(subset=[target_column])

In [16]:
if "totalcharges" in df.columns :
  df["Totalcharges"] = pd.to_numeric(df["Totalcharges"],errors="coerce")
  df["TotalCharges"].fillna(df["TotalCharges"].median(), inplace=True)

In [20]:
X = df .drop (columns=[target_column])
Y = df [target_column]

In [21]:
for col in X.columns:
    if X[col].dtype != 'O':
        X[col].fillna(X[col].median(), inplace=True)
    else:
        X[col].fillna(X[col].mode()[0], inplace=True)


/tmp/ipython-input-3884063407.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(X[col].mode()[0], inplace=True)
/tmp/ipython-input-3884063407.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [22]:
X_train , X_Test , Y_Train , Y_Test = train_test_split(
 X , Y ,
  test_size = 0.2 , stratify = Y , random_state = 42)

In [27]:
print("\nTrain shape:", X_train.shape)
print("Test shape:", X_Test.shape)


Train shape: (5634, 20)
Test shape: (1409, 20)


In [28]:
categorical_cols = [col for col in X.columns if X[col].dtype == 'O']
numeric_cols = [col for col in X.columns if col not in categorical_cols]



In [29]:
print("\nCategorical columns:", categorical_cols)
print("Numeric columns:", numeric_cols)


Categorical columns: ['customerID', 'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'TotalCharges']
Numeric columns: ['SeniorCitizen', 'tenure', 'MonthlyCharges']


In [30]:
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

In [31]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_cols),
        ("num", "passthrough", numeric_cols)
    ]
)

In [32]:
log_reg_model = LogisticRegression(max_iter=1000)


In [33]:
clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", log_reg_model)
])

In [35]:
clf.fit(X_train, Y_Train)
print("\nModel training completed.")



Model training completed.


In [37]:
y_pred = clf.predict(X_Test)

In [39]:
accuracy = accuracy_score(Y_Test, y_pred)
print("\nAccuracy on test set:", round(accuracy, 4))


Accuracy on test set: 0.7949


In [41]:
cm = confusion_matrix(Y_Test, y_pred)
print("\nConfusion Matrix:")
print(cm)


Confusion Matrix:
[[918 117]
 [172 202]]


In [43]:
print("\nClassification Report:")
print(classification_report(Y_Test, y_pred))



Classification Report:
              precision    recall  f1-score   support

          No       0.84      0.89      0.86      1035
         Yes       0.63      0.54      0.58       374

    accuracy                           0.79      1409
   macro avg       0.74      0.71      0.72      1409
weighted avg       0.79      0.79      0.79      1409

